In [1]:
import joblib
import numpy as np
import pandas as pd
import gc
import time
import os
import sys
from contextlib import contextmanager
from lightgbm import LGBMClassifier
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import  f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from optbinning import OptimalBinning
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA
from functools import reduce
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import optuna
from optuna import Trial
from sklearn.metrics import f1_score
from optuna.samplers import TPESampler
sampler = TPESampler(seed=42)


(CVXPY) Jan 31 09:45:40 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 31 09:45:40 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:

pd.set_option('display.max_rows', 500)
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [16]:
# read data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')
original_columns = test.columns.tolist()

In [17]:
import re
def extract_years(s):
    if "unknown" in s.lower():
        return np.nan  # or return some specific value
    elif "<" in s:
        return 0.5
    elif "+" in s :
        return 12 
    else:
        return int(re.search(r'\d+', s).group())
    
train['근로기간']= train['근로기간'].apply(extract_years)
test['근로기간']= test['근로기간'].apply(extract_years)

train['대출기간'] = train['대출기간'].apply(lambda x : int(x.split()[0]))
test['대출기간'] = test['대출기간'].apply(lambda x : int(x.split()[0]))


In [18]:
grade_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6}
column_dict = {
    '대출금액': 'amount',
    '대출기간': 'term',
    '근로기간': 'workperiod',
    '주택소유상태': 'homeown',
    '연간소득': 'income',
    '부채_대비_소득_비율': 'detinratio',
    '총계좌수': 'account_cnt',
    '대출목적': 'purpose',
    '최근_2년간_연체_횟수': 'delay_cnt',
    '총상환원금': 'principal_paid',
    '총상환이자': 'interest_paid',
    '총연체금액': 'delinquent_amount',
    '연체계좌수': 'delinquent_accounts',
    '대출등급': 'grade'
}

train['대출등급'] = train['대출등급'].map(grade_dict)

train = train.rename(columns=column_dict)
test = test.rename(columns=column_dict)

In [19]:
import category_encoders as ce
def target_encode_multiclass_train(X,y): #X,y are pandas df and series
    y=y.astype(str)   #convert to string to onehot encode
    enc=ce.OneHotEncoder().fit(y)
    y_onehot=enc.transform(y)
    class_names=y_onehot.columns  #names of onehot encoded columns
    X_obj=X.select_dtypes('object') #separate categorical columns
    X=X.select_dtypes(exclude='object') 

    encoders = {}  # store encoders for each class
    for class_ in class_names:
        enc=ce.TargetEncoder()
        enc.fit(X_obj,y_onehot[class_]) #convert all categorical columns for class_
        encoders[class_] = enc
        temp=enc.transform(X_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        X=pd.concat([X,temp],axis=1)    #add to original dataset
    return X, encoders

def target_encode_multiclass_test(X, encoders): #X is pandas df, encoders is dict of encoders
    X_obj=X.select_dtypes('object') #separate categorical columns
    X=X.select_dtypes(exclude='object') 

    for class_, enc in encoders.items():
        temp=enc.transform(X_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        X=pd.concat([X,temp],axis=1)    #add to original dataset
    return X


In [20]:
median_train = train['workperiod'].median()
train['homeown'] = train['homeown'].replace('ANY', 'MORTGAGE')
train['workperiod'] = train['workperiod'].fillna(median_train)
test['workperiod'] = test['workperiod'].fillna(median_train)

In [21]:
# 'train' 데이터프레임의 칼럼 이름을 문자열로 변환
train.columns = train.columns.map(str)
test.columns = test.columns.map(str)

x = train.drop(['ID','grade'], axis=1 )
y = train['grade']
test  = test.drop('ID', axis=1)

In [23]:
x,t_encoder =target_encode_multiclass_train(x,y)
test = target_encode_multiclass_test(test,t_encoder)

In [24]:
test

,amount,term,workperiod,income,detinratio,account_cnt,delay_cnt,principal_paid,interest_paid,delinquent_amount,...,homeown_grade_3,purpose_grade_3,homeown_grade_4,purpose_grade_4,homeown_grade_5,purpose_grade_5,homeown_grade_6,purpose_grade_6,homeown_grade_7,purpose_grade_7
0,16800000,36,8.0,132000000,19.64,12,0,394692,146604.0,0.0,...,0.201106,0.198864,0.128570,0.130682,0.018421,0.023539,0.003588,0.004221,0.072014,0.068994
1,8400000,36,5.0,89971200,15.84,25,0,0,0.0,0.0,...,0.139716,0.145712,0.149927,0.154651,0.022676,0.023772,0.005172,0.004823,0.081050,0.090281
2,17280000,36,6.0,150000000,8.41,20,0,1786980,281820.0,0.0,...,0.139716,0.262204,0.149927,0.085388,0.022676,0.005592,0.005172,0.000694,0.081050,0.035796
3,14400000,36,5.0,66000000,13.72,30,1,669024,281724.0,0.0,...,0.201106,0.262204,0.128570,0.085388,0.018421,0.005592,0.003588,0.000694,0.072014,0.035796
4,27600000,36,5.0,55200000,30.50,12,0,1250052,614844.0,0.0,...,0.139716,0.262204,0.149927,0.085388,0.022676,0.005592,0.005172,0.000694,0.081050,0.035796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,30000000,36,3.0,78000000,22.08,27,2,1307532,763380.0,0.0,...,0.201106,0.145712,0.128570,0.154651,0.018421,0.023772,0.003588,0.004823,0.072014,0.090281
64193,30000000,60,12.0,109200000,12.06,26,0,960612,1245252.0,0.0,...,0.201106,0.145712,0.128570,0.154651,0.018421,0.023772,0.003588,0.004823,0.072014,0.090281
64194,6120000,36,12.0,39600000,28.80,33,0,131520,80880.0,0.0,...,0.139716,0.145712,0.149927,0.154651,0.022676,0.023772,0.005172,0.004823,0.081050,0.090281
64195,11520000,36,12.0,66000000,25.44,41,1,1339536,601872.0,0.0,...,0.201106,0.145712,0.128570,0.154651,0.018421,0.023772,0.003588,0.004823,0.072014,0.090281


In [25]:
from sklearn.ensemble import VotingClassifier,BaggingClassifier
lgbm= LGBMClassifier( random_state=42)
xgb =XGBClassifier(random_state =42)
bag = BaggingClassifier(random_state=42)
voting_clf = VotingClassifier(
    estimators=[('lgbm', lgbm), ('xgb', xgb), ('bag', bag) ],
    voting='soft',  weights=[0.2,0.6,0.3]
)
# 모델 학습
voting_clf.fit(x, y)

# 예측
pred = voting_clf.predict(test)

In [8]:
for var in ['income','account_cnt','principal_paid','interest_paid','delinquent_amount']:
    train[var+'_log'] = np.log(train[var]+1)
    test[var+'_log'] = np.log(test[var]+1)

/tmp/ipykernel_13169/1543345230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[var+'_log'] = np.log(train[var]+1)
/tmp/ipykernel_13169/1543345230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[var+'_log'] = np.log(train[var]+1)
/tmp/ipykernel_13169/1543345230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [43]:
# from sklearn.preprocessing import MinMaxScaler

# # 스케일링할 열 선택
# columns_to_scale = ['income', 'principal_paid', 'interest_paid','account_cnt']
# # MinMaxScaler 객체 생성
# scaler = MinMaxScaler()

# # 훈련 데이터에 대해 fit과 transform 수행
# train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
# test[columns_to_scale] = scaler.transform(test[columns_to_scale])


In [10]:
# Group Operation
for group in ['term', 'homeown', 'purpose' ]:
    for num_var in ['amount', 'income','detinratio','delay_cnt','principal_paid','interest_paid','delinquent_amount']:
      
        train,test= train.copy(), test.copy()
        
        for stat in ['mean', 'median']:
            stat_dict = train.groupby([group])[num_var].agg(stat).to_dict()
            
            train[num_var +'_'+group + '_'+ stat]= train[group].map(stat_dict)
            test[num_var +'_'+group + '_'+ stat]= test[group].map(stat_dict)
            
            train[num_var+'_minus_'+num_var+'_'+group+'_'+stat] = train[num_var] - train[num_var+'_'+group+'_'+stat]
            test[num_var+'_minus_'+num_var+'_'+group+'_'+stat] = test[num_var] - test[num_var+'_'+group+'_'+stat]
            
            train[num_var+'_divide_'+num_var+'_'+group+'_'+stat] = np.log(train[num_var]+0.00001) / np.log(train[num_var+'_'+group+'_'+stat]+0.00001)
            test[num_var+'_divide_'+num_var+'_'+group+'_'+stat] = np.log(test[num_var]+0.00001) / np.log(test[num_var+'_'+group+'_'+stat]+0.00001)

In [11]:
# Woe Features
from optbinning import OptimalBinning
for variable in ['amount','income','detinratio','principal_paid','interest_paid','term']:

    optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

    x = train[variable].values
    y = train['grade']

    optb.fit(x, y)

    binning_table = optb.binning_table

    a1 = binning_table.build()
    
    train[variable+'_woe'] = pd.cut(train[variable],bins=optb.splits.tolist()).astype(str)
    test[variable+'_woe'] = pd.cut(test[variable],bins=optb.splits.tolist()).astype(str)

In [13]:
from sklearn.cluster import KMeans

# 클러스터링할 피처들을 지정합니다.
features = ['amount','income','detinratio','interest_paid']

# 최적의 클러스터 수로 KMeans 모델을 생성하고 학습합니다.
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=800, n_init=25, random_state=42)
kmeans.fit(train[features])

# 학습 데이터에 클러스터 번호를 추가합니다.
train['clust_num'] = kmeans.predict(train[features])

# 동일한 모델을 사용해 테스트 데이터에도 클러스터 번호를 추가합니다.
test['clust_num'] = kmeans.predict(test[features])


for var in ['income','principal_paid','interest_paid']:
    a1 = train.groupby(['clust_num'])[var].mean().to_dict() 
    train['clnum_'+var] = train['clust_num'].map(a1)
    test['clnum_'+var] = test['clust_num'].map(a1)
    
    a1 = train.groupby(['clust_num'])[var].min().to_dict()
    train['clnum2_'+var]= train['clust_num'].map(a1)
    test['clnum2_'+var]= test['clust_num'].map(a1)
    
    

In [14]:
#인코딩 처리하기 
from sklearn.preprocessing import OneHotEncoder

test['purpose'] = test['purpose'].replace('결혼', '기타')
# # 원-핫 인코더 생성
# encoder = OneHotEncoder(sparse=False)

encoder = LabelEncoder() 
# 'amount','income','detinratio','principal_paid','interest_paid','term'
for column in ['homeown','purpose','amount_woe','term_woe','income_woe','detinratio_woe','principal_paid_woe','interest_paid_woe']:
    # train 데이터를 기반으로 인코딩을 학습하고 적용
    encoder.fit(train[column])
    train[column] = encoder.transform(train[column])
    test[column] = encoder.transform(test[column])

In [15]:
# feature making
from itertools import chain, product 
# candidate_var = ['detinratio','amount','term','income','account_cnt','principal_paid','interest_paid'],['amount','workperiod','income','detinratio','account_cnt','principal_paid','interest_paid']
candidate_var =['amount','workperiod','income','detinratio','account_cnt','principal_paid','interest_paid']
pairs = list(chain(product(candidate_var, candidate_var), product(candidate_var, candidate_var))) 
pairs = pd.Series([sorted([i,j]) for (i,j) in set(pairs) if i!=j]).drop_duplicates().reset_index(drop=True).tolist()
pairs = sorted(pairs)

epsilon = 0.001  # 분모에 더해줄 작은 값

for i in range(len(pairs)):
    train[pairs[i][0]+'M'+pairs[i][1]] = train[pairs[i][0]] * train[pairs[i][1]]
    test[pairs[i][0]+'M'+pairs[i][1]] = test[pairs[i][0]] * test[pairs[i][1]]
    
    # 분모에 작은 값을 더해서 나눗셈 수행
    train[pairs[i][0]+'D'+pairs[i][1]] = train[pairs[i][0]] / (train[pairs[i][1]] + epsilon)
    test[pairs[i][0]+'D'+pairs[i][1]] = test[pairs[i][0]] / (test[pairs[i][1]] + epsilon)

    # 분모가 0인 경우를 대비해 조건문 수정
    train.loc[train[pairs[i][1]] == 0, pairs[i][0]+'D'+pairs[i][1]] = train.loc[train[pairs[i][1]] == 0, pairs[i][0]] / epsilon
    test.loc[test[pairs[i][1]] == 0, pairs[i][0]+'D'+pairs[i][1]] = test.loc[test[pairs[i][1]] == 0, pairs[i][0]] / epsilon



In [17]:
# 'train' 데이터프레임의 칼럼 이름을 문자열로 변환
train.columns = train.columns.map(str)
test.columns = test.columns.map(str)

x = train.drop(['ID','grade'], axis=1 )
y = train['grade']
test  = test.drop('ID', axis=1)

In [51]:
#Sampling
from imblearn.over_sampling import ADASYN

x,y = ADASYN(random_state = 42 ).fit_resample(x,y) 

In [21]:
# 데이터를 훈련 세트와 테스트 세트로 분리
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)


# Model Train

In [22]:
# XGBoost Optuna


def macro_f1_score(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    return f1  # Optuna는 최대화를 목표로 하므로 1 - f1이 아닌 f1을 반환


def objective(trial, train_x, train_y, val_x, val_y):
    param = {
        'objective': 'multi:softmax',  # 목표를 다중 클래스 분류로 설정
        'n_estimators': trial.suggest_int('n_estimators', 170, 1000),
        'learning_rate': trial.suggest_float('learning_rate',0.0005, 0.15),
        'lambda': trial.suggest_float('lambda', 1e-3, 0.1),
        'alpha': trial.suggest_float('alpha', 1e-3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1),
        'subsample': trial.suggest_float('subsample', 0.4, 1),
        'max_depth': trial.suggest_int('max_depth',2,10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
    }
    model =XGBClassifier(**param)  
    model.fit(train_x, train_y)
    preds = model.predict(val_x)
    score = macro_f1_score(val_y, preds)
    return score
          
# study_xgb.optimize(objective, n_trials=5)
study =  optuna.create_study(study_name='Xgb', direction='maximize',sampler=TPESampler(seed=42) )
study.optimize(lambda trial: objective(trial,X_train,  y_train ,X_valid , y_valid ),n_trials=14, timeout = 8300 )
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

# 기존 특성들 :[I 2024-01-22 14:42:19,100] Trial 8 finished with value: 0.9374472481030386 and parameters: {'n_estimators': 353, 'learning_rate': 0.08163306443215815, 'lambda': 0.014951498272501501, 'alpha': 0.8023947837732857, 'colsample_bytree': 0.44473038620786254, 'subsample': 0.9921321619603104, 'max_depth': 8, 'min_child_weight': 10}. Best is trial 8 with value: 0.9374472481030386.
# 새 특성+ 분산0인것 제외 :  and parameters: {'n_estimators': 752, 'learning_rate': 0.08513602429461944, 'lambda': 0.03391515402646526, 'alpha': 0.6532768202909376, 'colsample_bytree': 0.4470774309800684, 'subsample': 0.7236838914160535, 'max_depth': 8, 'min_child_weight': 1}. Best is trial 13 with value: 0.937929329398787.

[I 2024-01-31 18:11:36,383] A new study created in memory with name: Xgb
[I 2024-01-31 18:13:40,526] Trial 0 finished with value: 0.9070487314183808 and parameters: {'n_estimators': 481, 'learning_rate': 0.14263178880828245, 'lambda': 0.07346740023932911, 'alpha': 0.5990598257128396, 'colsample_bytree': 0.4936111842654619, 'subsample': 0.49359671220172163, 'max_depth': 2, 'min_child_weight': 44}. Best is trial 0 with value: 0.9070487314183808.
[I 2024-01-31 18:20:35,403] Trial 1 finished with value: 0.9254736490135481 and parameters: {'n_estimators': 669, 'learning_rate': 0.1063568503805088, 'lambda': 0.0030378649352844423, 'alpha': 0.9699399423098324, 'colsample_bytree': 0.899465584480253, 'subsample': 0.5274034664069657, 'max_depth': 3, 'min_child_weight': 10}. Best is trial 1 with value: 0.9254736490135481.
[I 2024-01-31 18:25:52,949] Trial 2 finished with value: 0.9173360965484878 and parameters: {'n_estimators': 422, 'learning_rate': 0.07895108652901955, 'lambda': 0.04376255684556

KeyboardInterrupt: 

In [18]:
from xgboost import XGBClassifier

# [I 2024-01-29 14:58:47,402] A new study created in memory with name: Xgb
# [I 2024-01-29 15:01:34,810] Trial 0 finished with value: 0.950629624448874 and parameters: {'n_estimators': 481, 'learning_rate': 0.14263178880828245, 'lambda': 0.07346740023932911, 'alpha': 0.5990598257128396, 'colsample_bytree': 0.4936111842654619, 'subsample': 0.49359671220172163, 'max_depth': 2, 'min_child_weight': 44}. Best is trial 0 with value: 0.950629624448874.
# [I 2024-01-29 15:10:54,948] Trial 1 finished with value: 0.9628777155557814 and parameters: {'n_estimators': 669, 'learning_rate': 0.1063568503805088, 'lambda': 0.0030378649352844423, 'alpha': 0.9699399423098324, 'colsample_bytree': 0.899465584480253, 'subsample': 0.5274034664069657, 'max_depth': 3, 'min_child_weight': 10}. Best is trial 1 with value: 0.9628777155557814.
# [W 2024-01-29 15:16:16,162] Trial 2 failed with parameters: {'n_estimators': 422, 'learning_rate': 0.07895108652901955, 'lambda': 0.04376255684556946, 'alpha': 0.2919379110578439, 'colsample_bytree': 0.7671117368334277, 'subsample': 0.4836963163912251, 'max_depth': 4, 'min_child_weight': 19} because of the following error: KeyboardInterrupt().
# estimators': 669, 'learning_rate': 0.1063568503805088, 'lambda': 0.0030378649352844423, 'alpha': 0.9699399423098324, 'colsample_bytree': 0.899465584480253, 'subsample': 0.5274034664069657, 'max_depth': 3, 'min_child_weight': 10}. Best is trial 1 with value: 0.9254736490135481.

# Traceback (most recent call last):

# 파라미터 설정
params = {'n_estimators': 669, 
          'learning_rate': 0.1063568503805088, 
          'lambda': 0.0030378649352844423, 
          'alpha': 0.9699399423098324, 
          'colsample_bytree': 0.899465584480253, 
          'subsample': 0.5274034664069657, 
          'max_depth': 3, 
          'min_child_weight': 10}

# XGBClassifier에 파라미터 설정
xgb = XGBClassifier(**params)

# 모델 학습
xgb.fit(x, y)

# 예측
pred = xgb.predict(test)

In [26]:

sample_submission['대출등급'] = pred 

# 딕셔너리 뒤집기
inv_grade_dict = {v: k for k, v in grade_dict.items()}

# 'grade' 열을 다시 알파벳으로 변경
sample_submission['대출등급'] = sample_submission['대출등급'].map(inv_grade_dict)
sample_submission

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,D
64193,TEST_64193,D
64194,TEST_64194,D
64195,TEST_64195,C


In [27]:
sample_submission.to_csv('../data/targetencoding1.csv', index= False)